# 🧪 Executar Testes do Projeto

Este notebook permite executar todos os testes do projeto MLOps de forma interativa.

## Estrutura de Testes

- **Testes Unitários** (`tests/unit/`): Testes rápidos e isolados (55 arquivos)
- **Testes de Integração** (`tests/integration/`): Testes end-to-end (7 arquivos)
- **Total**: 62 arquivos de teste
- **Cobertura**: ~85%

## 📦 Setup - Imports e Configuração

In [18]:
import sys
from pathlib import Path
import subprocess
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path

# Define o diretório raiz do projeto
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"📁 Diretório do projeto: {PROJECT_ROOT}")
print(f"📂 Diretório de testes: {PROJECT_ROOT / 'tests'}")

📁 Diretório do projeto: c:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops
📂 Diretório de testes: c:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops\tests


## Executar Todos os Testes

Executa todos os testes (unitários + integração).

In [ ]:
%%time
# Executa todos os testes
tests_path = str(PROJECT_ROOT / 'tests')
!python -m pytest "{tests_path}" -v --tb=short

In [ ]:
%%time
# Executa apenas testes unitários
tests_path = str(PROJECT_ROOT / 'tests' / 'unit')
!python -m pytest "{tests_path}" -v --tb=short

## Análise Detalhada dos Resultados

Gera uma tabela detalhada com todos os resultados dos testes.

In [19]:
# Cria pasta de relatórios
relatorios_dir = PROJECT_ROOT / 'relatorios'
relatorios_dir.mkdir(exist_ok=True)

# Paths completos como na célula 7
tests_path = str(PROJECT_ROOT / 'tests')
xml_path = str(relatorios_dir / 'test_results.xml')

# Executa pytest igual à célula 7 que funciona
result = subprocess.run(
    f'python -m pytest "{tests_path}" --junit-xml="{xml_path}" -v',
    cwd=PROJECT_ROOT,
    capture_output=True,
    text=True,
    shell=True
)

# Verifica se o arquivo foi criado
if not (relatorios_dir / 'test_results.xml').exists():
    display(pd.DataFrame([{
        'Erro': 'Nenhum teste foi executado',
        'Detalhes': 'Verifique se há testes válidos no diretório tests/',
        'Stdout': result.stdout[:300] if result.stdout else 'Vazio',
        'Stderr': result.stderr[:300] if result.stderr else 'Vazio'
    }]))
else:
    # Parse XML
    tree = ET.parse(relatorios_dir / 'test_results.xml')
    root = tree.getroot()
    
    data = []
    for testcase in root.iter('testcase'):
        file_path = testcase.get('file', '')
        name = testcase.get('name', '')
        
        # Status
        if testcase.find('failure') is not None:
            status = 'FALHOU'
            msg = testcase.find('failure').get('message', '')[:100]
        elif testcase.find('error') is not None:
            status = 'ERRO'
            msg = testcase.find('error').get('message', '')[:100]
        elif testcase.find('skipped') is not None:
            status = 'PULOU'
            msg = testcase.find('skipped').get('message', '')[:100]
        else:
            status = 'PASSOU'
            msg = ''
        
        # Módulo
        if 'tests/unit/' in file_path or 'tests\\unit\\' in file_path:
            path_clean = file_path.replace('tests/unit/', '').replace('tests\\unit\\', '')
            modulo = path_clean.split('/')[0].split('\\')[0]
        elif 'tests/integration/' in file_path or 'tests\\integration\\' in file_path:
            modulo = 'integration'
        else:
            modulo = 'root'
        
        arquivo = file_path.split('/')[-1].split('\\')[-1] if file_path else ''
        
        data.append({
            'Módulo': modulo,
            'Arquivo': arquivo,
            'Teste': name,
            'Status': status,
            'Detalhes': msg
        })
    
    df = pd.DataFrame(data)
    
    # Resumo
    resumo = pd.DataFrame([{
        'Total': len(df),
        'Passou': len(df[df['Status'] == 'PASSOU']),
        'Falhou': len(df[df['Status'] == 'FALHOU']),
        'Erro': len(df[df['Status'] == 'ERRO']),
        '⏭Pulou': len(df[df['Status'] == '⏭PULOU'])
    }])
    
    # Por módulo
    por_modulo = df.groupby(['Módulo', 'Status']).size().unstack(fill_value=0)
    
    # Problemas
    problemas = df[df['Status'].isin(['FALHOU', 'ERRO'])][['Módulo', 'Arquivo', 'Teste', 'Status', 'Detalhes']]
    
    # Salva CSVs
    df.to_csv(relatorios_dir / 'resultados_completos.csv', index=False)
    resumo.to_csv(relatorios_dir / 'resumo.csv', index=False)
    por_modulo.to_csv(relatorios_dir / 'por_modulo.csv')
    if len(problemas) > 0:
        problemas.to_csv(relatorios_dir / 'problemas.csv', index=False)
    
    # Display tables
    display(resumo)
    display(por_modulo)
    if len(problemas) > 0:
        display(problemas)

Exception in thread Thread-32 (_readerthread):
Traceback (most recent call last):
  File "C:\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Python311\Lib\subprocess.py", line 1550, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Python311\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 49813: character maps to <undefined>


,Total,Passou,Falhou,Erro,⏭Pulou
0,302,177,101,12,0


Status,ERRO,FALHOU,PASSOU,PULOU
Módulo,,,,
root,12,101,177,12


,Módulo,Arquivo,Teste,Status,Detalhes
4,root,,test_pipeline_com_dados_reais_subset,FALHOU,"AssertionError: assert 'imc' in Index(['data',..."
8,root,,test_artefatos_codificacao_criados,FALHOU,AssertionError: assert 'artefatos_codificacao'...
9,root,,test_artefatos_normalizacao_criados,FALHOU,AssertionError: assert 'artefatos_normalizacao...
10,root,,test_mesmo_dataset_ambos_tipos,FALHOU,TypeError: treinar_rapido() got an unexpected ...
11,root,,test_pipeline_completo_classificacao,FALHOU,AssertionError: assert 'imc' in Index(['idade'...
...,...,...,...,...,...
254,root,,test_treinar_modelo_basico,FALHOU,AttributeError: 'RandomForestClassifier' objec...
255,root,,test_modelo_treinado_faz_predicoes,FALHOU,AttributeError: 'LogisticRegression' object ha...
256,root,,test_modelo_com_diferentes_tipos,FALHOU,AttributeError: 'RandomForestClassifier' objec...
273,root,,test_extrai_estimador_pycaret,FALHOU,AssertionError: assert <treinamento.utils.test...


## 🔗 Executar Apenas Testes de Integração

Testes de integração são mais lentos e testam fluxos completos end-to-end.

In [15]:
%%time
# Executa apenas testes de integração
tests_path = str(PROJECT_ROOT / 'tests' / 'integration')
!python -m pytest "{tests_path}" -v --tb=short

============================= test session starts =============================
platform win32 -- Python 3.11.0, pytest-7.4.3, pluggy-1.6.0 -- c:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops\.venv\Scripts\python.exe
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase(WindowsPath('c:/Arquivos_Pessoais/PROJETOS/tcc_mba_esalq_mlops/notebooks/.hypothesis/examples'))
rootdir: c:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops
configfile: pytest.ini
plugins: anyio-4.12.1, dash-4.0.0, Faker-20.1.0, hypothesis-6.99.13, asyncio-0.21.1, cov-4.1.0, mock-3.12.0, timeout-2.4.0
asyncio: mode=Mode.STRICT
collecting ... collected 12 items / 4 skipped

..\tests\integration\test_cenarios_reais.py::TestCenarioRealConfortoTermico::test_pipeline_com_dados_reais_subset FAILED [  8%]
..\tests\integration\test_cenarios_reais.py::TestCenariosEdgeCases::test_dados_com_muitos_missings PASSED [ 16%]
..\tests\integration\test_cenarios_reais.py::TestCenariosEdgeCases::tes

## 🎯 Executar Testes de Módulos Específicos

### Testes de Features

In [ ]:
# Testa módulo de features (codificação, normalização, criação)
tests_path = str(PROJECT_ROOT / 'tests' / 'unit' / 'features')
!python -m pytest "{tests_path}" -v

: 

### Testes de Treinamento

In [ ]:
# Testa módulo de treinamento
tests_path = str(PROJECT_ROOT / 'tests' / 'unit' / 'treinamento')
!python -m pytest "{tests_path}" -v

### Testes de Pipelines

In [ ]:
# Testa pipelines (processamento, features, treinamento)
tests_path = str(PROJECT_ROOT / 'tests' / 'unit' / 'pipelines')
!python -m pytest "{tests_path}" -v

### Testes de Processamento

In [ ]:
# Testa processamento (limpeza, imputação, temporal)
tests_path = str(PROJECT_ROOT / 'tests' / 'unit' / 'processamento')
!python -m pytest "{tests_path}" -v

## 📊 Relatório de Cobertura de Código

Gera relatório HTML mostrando quais linhas de código estão cobertas por testes.

In [16]:
%%time
# Cria pasta de relatórios
relatorios_dir = PROJECT_ROOT / 'relatorios'
relatorios_dir.mkdir(exist_ok=True)

# Executa testes com relatório de cobertura
tests_path = str(PROJECT_ROOT / 'tests')
src_path = str(PROJECT_ROOT / 'src')
htmlcov_path = str(relatorios_dir / 'htmlcov')

!python -m pytest "{tests_path}" --cov="{src_path}" --cov-report=html:"{htmlcov_path}" --cov-report=term

print(f"\n✅ Relatório HTML gerado em: relatorios/htmlcov/index.html")

============================= test session starts =============================
platform win32 -- Python 3.11.0, pytest-7.4.3, pluggy-1.6.0 -- c:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops\.venv\Scripts\python.exe
cachedir: .pytest_cache
hypothesis profile 'default' -> database=DirectoryBasedExampleDatabase(WindowsPath('c:/Arquivos_Pessoais/PROJETOS/tcc_mba_esalq_mlops/notebooks/.hypothesis/examples'))
rootdir: c:\Arquivos_Pessoais\PROJETOS\tcc_mba_esalq_mlops
configfile: pytest.ini
plugins: anyio-4.12.1, dash-4.0.0, Faker-20.1.0, hypothesis-6.99.13, asyncio-0.21.1, cov-4.1.0, mock-3.12.0, timeout-2.4.0
asyncio: mode=Mode.STRICT
collecting ... collected 298 items / 4 skipped

..\tests\integration\test_cenarios_reais.py::TestCenarioRealConfortoTermico::test_pipeline_com_dados_reais_subset FAILED [  0%]
..\tests\integration\test_cenarios_reais.py::TestCenariosEdgeCases::test_dados_com_muitos_missings PASSED [  0%]
..\tests\integration\test_cenarios_reais.py::TestCenariosEdgeCases::te

## 📝 Resumo Rápido dos Testes

Executa testes e mostra apenas o resumo final.

In [ ]:
# Executa testes mostrando apenas resumo (sem verbose)
tests_path = str(PROJECT_ROOT / 'tests')
!python -m pytest "{tests_path}" --tb=line -q

## 🔍 Executar Teste Específico

Para executar um teste específico, use o padrão:
```
pytest caminho/arquivo.py::nome_da_funcao_teste
```

In [ ]:
# Exemplo: executar teste específico de normalização
test_file = str(PROJECT_ROOT / 'tests' / 'unit' / 'features' / 'normalizacao' / 'test_normalizar.py')
!python -m pytest "{test_file}::test_normalizar_standard" -v

## 🐛 Executar Testes com Debug

Para investigar falhas, use modo verbose com stack trace completo.

In [ ]:
# Executa com output completo e para no primeiro erro
tests_path = str(PROJECT_ROOT / 'tests' / 'unit')
!python -m pytest "{tests_path}" -v -s -x --tb=long

## 📈 Executar Testes End-to-End Principais

Executa apenas os testes de integração mais importantes.

In [12]:
%%time
# Testa pipeline end-to-end completo
test_file = str(PROJECT_ROOT / 'tests' / 'integration' / 'test_pipeline_end_to_end.py')
!pytest {test_file} -v

CPU times: total: 0 ns
Wall time: 279 ms


Failed to canonicalize script path


In [ ]:
%%time
# Testa cenários reais e edge cases
test_file = str(PROJECT_ROOT / 'tests' / 'integration' / 'test_cenarios_reais.py')
!pytest {test_file} -v

## 🎨 Estatísticas dos Testes

Conta quantos testes existem por categoria.

In [6]:
from pathlib import Path

tests_dir = PROJECT_ROOT / 'tests'

# Conta testes por categoria
unit_tests = list((tests_dir / 'unit').rglob('test_*.py'))
integration_tests = list((tests_dir / 'integration').rglob('test_*.py'))

print("📊 ESTATÍSTICAS DE TESTES")
print("=" * 50)
print(f"\n✅ Testes Unitários: {len(unit_tests)} arquivos")
print(f"   • Features: {len(list((tests_dir / 'unit' / 'features').rglob('test_*.py')))} testes")
print(f"   • Processamento: {len(list((tests_dir / 'unit' / 'processamento').rglob('test_*.py')))} testes")
print(f"   • Treinamento: {len(list((tests_dir / 'unit' / 'treinamento').rglob('test_*.py')))} testes")
print(f"   • Pipelines: {len(list((tests_dir / 'unit' / 'pipelines').rglob('test_*.py')))} testes")
print(f"   • Utils: {len(list((tests_dir / 'unit' / 'utils').rglob('test_*.py')))} testes")

print(f"\n🔗 Testes de Integração: {len(integration_tests)} arquivos")

print(f"\n📦 TOTAL: {len(unit_tests) + len(integration_tests)} arquivos de teste")
print("=" * 50)

📊 ESTATÍSTICAS DE TESTES

✅ Testes Unitários: 55 arquivos
   • Features: 12 testes
   • Processamento: 17 testes
   • Treinamento: 15 testes
   • Pipelines: 4 testes
   • Utils: 3 testes

🔗 Testes de Integração: 7 arquivos

📦 TOTAL: 62 arquivos de teste


## 🔥 Executar Teste Customizado

Use esta célula para executar comandos pytest personalizados.

---

## 📚 Comandos Úteis Pytest

### Opções Comuns
- `-v` : Verbose (mostra cada teste)
- `-s` : Mostra prints durante testes
- `-x` : Para no primeiro erro
- `-k "palavra"` : Executa apenas testes com "palavra" no nome
- `--tb=short` : Traceback curto
- `--tb=long` : Traceback completo
- `--tb=line` : Traceback de uma linha
- `-q` : Modo quieto (apenas resumo)
- `--maxfail=N` : Para após N falhas
- `--lf` : Executa apenas testes que falharam na última execução
- `--ff` : Executa testes que falharam primeiro

### Cobertura
- `--cov=src` : Mede cobertura do código em src/
- `--cov-report=html` : Gera relatório HTML
- `--cov-report=term` : Mostra relatório no terminal

### Marcadores
- `-m integration` : Executa apenas testes marcados como integração
- `-m "not slow"` : Pula testes marcados como lentos

### Exemplos
```bash
# Executar teste específico
pytest tests/unit/features/test_normalizar.py::test_normalizar_standard

# Executar testes que contém "pipeline" no nome
pytest -k "pipeline"

# Executar apenas testes que falharam
pytest --lf

# Executar com cobertura
pytest --cov=src --cov-report=html
```

# 2. Testes de Qualidade de Código